# Sudoku solver

Solve a 9x9 sudoku grid using linear programming.

In [91]:
from pulp import *
import itertools
import numpy as np
import csv

Set up problem details for a 9x9 grid.

In [36]:
n_range = list(range(1,10))
grid = list(itertools.product(n_range, n_range))

In [151]:
## Assign each square to a "sub-block" of squares
blocks = list(np.ceil(np.array(axis)/3.0).astype(int))
unique_blocks = set(blocks)
block_mapping = {b:[] for b in unique_blocks}
for i in axis:
    b = np.ceil(i/3.0)
    block_mapping[b].append(i)

Read the input problem.

In [113]:
file = open("../data/sudoku1.csv")
csvreader = csv.reader(file)
rows = []
for row in csvreader:
        rows.append(row)
file.close()
rows = rows[1:len(rows)]

In [169]:
input_dict = {}
input_arr = []
for x in n_range:
    temp_arr = []
    for y in n_range:
        if rows[x-1][y-1] != "":
            input_dict[(x,y)] = int(rows[x-1][y-1])
            temp_arr.append(int(rows[x-1][y-1]))
        else:
            temp_arr.append(0)
    input_arr.append(temp_arr)

Define the problem using PuLP.

In [152]:
# Create the 'prob' variable to contain the problem data
mdl = LpProblem("Sudoku",LpMinimize)

In [153]:
## Create the assignment variable - 9 x 9 x 9 (3d) matrix where x and y represent position and n represents
## the number chosen (1 means chosen, 0 means not)
choice = pulp.LpVariable.dicts("Choices", ((x, y, n) for (x in n_range for y in n_range for n in n_range)),
                              lowBound = 0,
                              upBound = 1)

In [154]:
## Dummy constraint (if the problem is solvable, there is exactly one feasible solution)
mdl += pulp.lpSum([choice[x, y, n] for (x in n_range for y in n_range for n in n_range)])

In [155]:
## Each number must exist only once in each row
for x in n_range:
    for n in n_range:
        cname = "Row {:d}, n {:d}".format(x, n)
        mdl += (
            pulp.lpSum([select[x, y, n] for y in n_range]) == 1,
            cname,
        )

In [156]:
## Each number must exist only once in each column
for y in n_range:
    for n in n_range:
        cname = "Col {:d}, n {:d}".format(y, n)
        mdl += (
            pulp.lpSum([select[x, y, n] for x in n_range]) == 1,
            cname,
        )

In [157]:
## Each number must exist only once in each sub-grid
for b_x in block_mapping.keys():
    for b_y in block_mapping.keys():
        for n in n_range:
            cname = "Block {:d}_{:d}, n {:d}".format(b_x, b_y, n)
            mdl += (
                pulp.lpSum([select[x, y, n] for x in block_mapping[b_x] for y in block_mapping[b_y]]) == 1,
                cname,
            )

In [158]:
## Inputs must be set as given
for x, y in input_dict.keys():
    n = input_dict[(x, y)]
    cname = "Input {:d}_{:d}, n {:d}".format(x, y, n)
    mdl += (select[x, y, n] == 1,  cname)

In [159]:
## Each square can only be assigned once
for x in n_range:
    for y in n_range:
        cname = "Square {:d}_{:d}".format(x, y)
        mdl += (
            pulp.lpSum([select[x, y, n] for n in n_range]) == 1,
            cname,
        )

Solve the problem.

In [160]:
mdl.solve()

1

In [161]:
print("Status:", LpStatus[mdl.status])

Status: Optimal


Extract results and examine solution.

In [162]:
# Each of the variables is printed with it's resolved optimum value
solution_dict = {}
solution_arr = []
for x in n_range:
    temp_arr = []
    for y in n_range:
        for n in n_range:
            v = pulp.value(select[x, y, n])
            if v == 1:
                solution_dict[(x, y)] = n
                temp_arr.append(n)
    solution_arr.append(temp_arr)

In [173]:
print("INPUT")
print(np.array(input_arr))
print("SOLUTION")
print(np.array(solution_arr))

INPUT
[[3 0 6 5 0 8 4 0 0]
 [5 2 0 0 0 0 0 0 0]
 [0 8 7 0 0 0 0 3 1]
 [0 0 3 0 1 0 0 8 0]
 [9 0 0 8 6 3 0 0 5]
 [0 5 0 0 9 0 6 0 0]
 [1 3 0 0 0 0 2 5 0]
 [0 0 0 0 0 0 0 7 4]
 [0 0 5 2 0 6 3 0 0]]
SOLUTION
[[3 1 6 5 7 8 4 9 2]
 [5 2 9 1 3 4 7 6 8]
 [4 8 7 6 2 9 5 3 1]
 [2 6 3 4 1 5 9 8 7]
 [9 7 4 8 6 3 1 2 5]
 [8 5 1 7 9 2 6 4 3]
 [1 3 8 9 4 7 2 5 6]
 [6 9 2 3 5 1 8 7 4]
 [7 4 5 2 8 6 3 1 9]]
